# LTR Example

I used this tutorial: https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/ltr.ipynb

In [1]:
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import pyterrier as pt
if not pt.started():
  pt.init()

In [3]:
dataset = pt.datasets.get_dataset("vaswani")

indexref = dataset.get_index()
topics = dataset.get_topics()
qrels = dataset.get_qrels()

In [4]:
BM25 = pt.BatchRetrieve(indexref, controls = {"wmodel": "BM25"})

TF_IDF =  pt.BatchRetrieve(indexref, controls = {"wmodel": "TF_IDF"})
PL2 =  pt.BatchRetrieve(indexref, controls = {"wmodel": "PL2"})
DirichletLM =  pt.BatchRetrieve(indexref, controls = {"wmodel": "DirichletLM"})
DPH =  pt.BatchRetrieve(indexref, controls = {"wmodel": "DPH"})

pipe = BM25 >> (pt.transformer.IdentityTransformer() ** TF_IDF ** PL2 ** DirichletLM ** DPH)
pipe = pipe.compile()

Applying 8 rules


In [5]:
train_topics, valid_topics, test_topics = np.split(topics, [int(.6*len(topics)), int(.8*len(topics))])

In [6]:
import xgboost as xgb
params = {'objective': 'rank:ndcg', 
          'learning_rate': 0.1, 
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,
          'verbose': 2,
          'random_state': 42 
         }

BaseLTR_LM = pipe >> pt.pipelines.XGBoostLTR_pipeline(xgb.sklearn.XGBRanker(**params))
BaseLTR_LM.fit(train_topics, qrels, valid_topics, qrels)

[21:02:35] WARNING: ../src/learner.cc:516: 
Parameters: { verbose } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [7]:
allresultsLM = pt.pipelines.Experiment([PL2, BaseLTR_LM],
                                test_topics,                                  
                                qrels, ["map"], 
                                names=["PL2 Baseline", "LambdaMART"])
# More features is not always better ;)
allresultsLM

,name,map
0,PL2 Baseline,0.206031
1,LambdaMART,0.190789
